<a href="https://colab.research.google.com/github/Vurimindi2021/Capstone3/blob/main/src/nlp_contradictory_Bert_Base_ML_Cased_KF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 2.2 MB/s 
     |████████████████████████████████| 895 kB 50.0 MB/s 
     |████████████████████████████████| 3.3 MB 72.6 MB/s 
     |████████████████████████████████| 636 kB 64.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, BertTokenizer, TFAutoModel, TFBertModel
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold,train_test_split
import os

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/nlp/data/train.csv")
test = pd.read_csv("/content/drive/MyDrive/nlp/data/test.csv")

In [ ]:
MAX_LENGTH=200
max_len=MAX_LENGTH*2

In [ ]:
def encode_sentence(s):
   return tokenizer.encode(s,padding='max_length', max_length=MAX_LENGTH,truncation=True)

In [ ]:
def bert_encode(premise, hypothesis, tokenizer):
    
    sentence1 = tf.ragged.constant([encode_sentence(s) for s in np.array(premise)])
    sentence2 = tf.ragged.constant([encode_sentence(s) for s in np.array(hypothesis)])
    
    #cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*(sentence1.shape[0])
    input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1)

    input_masks = tf.ones_like(input_word_ids).to_tensor()

    #type_cls = tf.zeros_like(cls)
    type_s1 = tf.zeros_like(sentence1)
    type_s2 = tf.ones_like(sentence2)
    input_type_ids = tf.concat([type_s1, type_s2], axis=-1).to_tensor()

    inputs = {
      'input_word_ids': input_word_ids.to_tensor(),
      'input_masks': input_masks,
      'input_type_ids': input_type_ids}
    
    return inputs

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
train_input = bert_encode(train.premise.values, train.hypothesis.values, tokenizer)
#train_label = tf.keras.utils.to_categorical(train.label.values)

In [ ]:
K.clear_session()

In [ ]:
def build_bert_baseline(max_len=max_len):
    bert_layer = TFBertModel.from_pretrained('bert-base-multilingual-cased')
    input_word_ids = tf.keras.Input(shape = (max_len,),dtype =tf.int32, name='input_word_ids')
    input_masks = tf.keras.Input(shape = (max_len,),dtype =tf.int32, name='input_masks')
    input_type_ids = tf.keras.Input(shape = (max_len,),dtype =tf.int32, name='input_type_ids')
    
    sequence_output = bert_layer([input_word_ids, input_masks, input_type_ids])[0]
    #sequence_output = clf_output[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(sequence_output[:,0,:])
    
    model = tf.keras.Model(inputs=[input_word_ids,input_masks,input_type_ids],outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=3e-5),
                  loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
    return model

In [ ]:
model = build_bert_baseline(max_len=max_len)
model.summary()

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 259)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 259)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 259)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_1 (TFBertModel)   TFBaseModelOutputWit 177853440   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
K.clear_session()

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Running on TPU ', tpu.master())
except ValueError:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Initializing the TPU system: grpc://10.123.30.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.123.30.242:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  grpc://10.123.30.242:8470
REPLICAS:  8


In [ ]:
def model_cv_score(model):    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=98)
    E_cv = 0
    for fold,(train_idx,val_idx) in enumerate(skf.split(train['premise'],train['label'])):
        print('\nFold %s:'% (fold))
        K.clear_session()

        train_premise = train['premise'].iloc[train_idx]
        train_hypothesis = train['hypothesis'].iloc[train_idx]
        train_label = train['label'].iloc[train_idx]

        val_premise = train['premise'].iloc[val_idx]
        val_hypothesis = train['hypothesis'].iloc[val_idx]
        val_label = train['label'].iloc[val_idx]
        
        ## use encode method you define, this method is for bert base and 
        ## subject to change when switched to roBerta
        cv_train_input = bert_encode(train_premise.values, train_hypothesis.values, tokenizer)
        cv_val_input = bert_encode(val_premise.values, val_hypothesis.values, tokenizer)

        #model = build_bert(max_len=55)
        filepath='/content/drive/MyDrive/nlp/check_point/bert_base_ml_cased_kf_fold%s.h5'%(fold)
        checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                                                    save_best_only=True, save_weights_only=True, 
                                                    mode='min',save_freq = 'epoch')
        model.fit(cv_train_input, train_label.values, epochs =2, 
              batch_size = 16, callbacks=[checkpoint], 
              validation_data=(cv_val_input,val_label.values))

        print('\nPredicting OOF Error......')
        model.load_weights('/content/drive/MyDrive/nlp/check_point/bert_base_ml_cased_kf_fold%s.h5'%(fold))
        val_pred = [np.argmax(p) for p in model.predict(cv_val_input)]
        oof_error = np.sum([pred!=true for pred,true in zip(val_pred,val_label)])/len(val_label)
        print('Fold_%s Error: %s' % (fold,oof_error))
        E_cv += oof_error

    print('\nE_cv = %s'%(E_cv/5))

In [ ]:
model_cv_score(model)